# Answering Business Questions Using SQL

In this project we're going to use SQL and the Chinook database to answer business questions. The Chinook database contains information about a fictional digital music shop. The tables within the database are as follows;

- employee
- customer
- invoice 
- invoice_line
- track
- playlist_track
- playlist
- media_type
- artist 
- album
- genre 

The SQL flavor that will be used throughout the project is SQLite. In order to run SQL queries in Jupyter we have to add %%sql on its own line before a query.

# Connect Jupyter Notebook to database file

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

# Display database tables

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Exploring employee, invoice, and track tables

In [4]:
%%sql
SELECT *
FROM employee
LIMIT 2;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com


In [5]:
%%sql
SELECT *
FROM invoice
LIMIT 2;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9


In [6]:
%%sql
SELECT *
FROM track
LIMIT 2;

Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99


# Finding which genres sell the most tracks in the United States of America

In [10]:
%%sql
WITH 
    usa AS
        (
            SELECT il.*
            FROM invoice_line il
            INNER JOIN invoice i on il.invoice_id = i.invoice_id
            INNER JOIN customer c on i.customer_id = c.customer_id
            WHERE c.country = 'USA'
        )
SELECT
    g.name genre,
    count(usa.invoice_line_id) tracks_sold,
    CAST(COUNT(usa.invoice_line_id) AS FLOAT) / (
            SELECT COUNT(*) from usa) percentage_sold
FROM usa usa
INNER JOIN track t ON t.track_id = usa.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY genre
ORDER BY tracks_sold DESC
LIMIT 10;    

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


The table above shows us how many tracks were sold in the USA by genre. We're tasked with determining which of the following four genres would sell the most tracks in the USA; Hip-Hop, Punk, Pop, or Blues. Let's list the four genres below with the total number of tracks sold for each;

- Hip-Hop: 20 tracks sold
- Punk: 130 tracks sold 
- Pop: 22 tracks sold
- Blues: 36 tracks sold

Next lets list the artists name and genres that we were are asked to select from;

- Regal - Hip-Hop
- Red Tone - Punk
- Meteor and the Girls - Pop
- Slim Jim Bites - Blues

Based on tracks sold in the USA, the three artists we should choose to add to our digital music shop that will most likely sell the most tracks should be; 

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

# Analyzing Sales Agents performance

In [15]:
%%sql

WITH 
    employee_sales AS
        (
            SELECT 
                i.customer_id,
                c.support_rep_id,
                SUM(i.total) total
            FROM invoice i
            INNER JOIN customer c ON i.customer_id = c.customer_id
            GROUP BY 1, 2
        )
SELECT 
    e.first_name || ' ' || e.last_name employee,
    e.hire_date,
    e.reports_to,
    SUM(es.total) total_sales
FROM employee_sales es
INNER JOIN employee e ON e.employee_id = es.support_rep_id
GROUP BY 1;

Done.


employee,hire_date,reports_to,total_sales
Jane Peacock,2017-04-01 00:00:00,2,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,2,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,2,1393.92


In [19]:
%%sql
SELECT 
    e.first_name || ' ' || e.last_name employee, 
    title,
    reports_to
FROM employee e;

Done.


employee,title,reports_to
Andrew Adams,General Manager,None
Nancy Edwards,Sales Manager,1
Jane Peacock,Sales Support Agent,2
Margaret Park,Sales Support Agent,2
Steve Johnson,Sales Support Agent,2
Michael Mitchell,IT Manager,1
Robert King,IT Staff,6
Laura Callahan,IT Staff,6


The first table above shows which employee had the highest total dollar amount in sales which is Jane Peacock. The hire date column was added in to determine how far apart each employee was hired, maybe one of them started at a later date but is a better sales person. From the results we can make the case that Steve Johnson performs best in sales since he started 6 months after Jane, 5 months after Margaret, and is already at 1393.92 in total sales - 337.58 behind Jane's 1731.50 in total sales. The reports to column was added in just to see if there was more than one sales manager which there is not, the only sales manager is Nancey Edwards and all sales support agents report to her. If there was multiple sales managers, we would be able to break down the total sales from each managers team and possibly uncover that one sales manager is better than another.

To reiterate, the total number for sales (in total dollar amount) for each rep is as follows;

- Jane Peacock: 1731.50
- Margaret Park: 1584.00
- Steve Johnson: 1393.92